In [2]:
pip install simpy

In [3]:
import simpy
import numpy 

0 - simple FIFO

In [100]:
#the order object that will be processed through the queue
class Order:
  def __init__(self, name, prep_time, arrival):
        self.name = name
        self.prep_time = prep_time
        self.arrival = arrival
        self.placement_time = None
        self.exit_time = None

def format_time(sim_time):
    hours, minutes = divmod(int(sim_time), 60)
    seconds = int((sim_time % 1) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#this function creates the orders and adds them to the queue
def order_generator(env, order_queue):
    order_count = 1
    while True:
        #set the minimum
        minimum = 2
        #generate a random preparation time (adding the 2 to make it the min)
        prep_time = round(minimum + numpy.random.exponential(scale=4.0, size=1)[0], 3)
        arrival = round(numpy.random.exponential(scale=4.0, size=1)[0], 3)
        #create an order with that preparation time
        order = Order(f"order{order_count}", prep_time, arrival)
        #increment the order counter
        order_count += 1

        #allowing some orders arrive at the same time
        #if this isn't satisfyed the next order will arive at the same time
        if (order_count % 4 != 0):
            #pauses this function for a random arrival time mimicking this part being idle
            yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])
        #pushes the order in the queue
        yield order_queue.put(order)
        #gives feedback that an order have arrived
        order.placement_time = env.now
        print(f"{format_time(env.now)}: {order.name} entered with prep time {order.prep_time} minutes.")

#this function process orders
def process_orders(env, order_queue):
    while True:
        #get the order that is on the top of the queue
        order = yield order_queue.get()
        #stores the time the order exists the queue
        order.exit_time = env.now
        #pauses this function for the preparation time, mimicking the restaurant creating the order
        yield env.timeout(order.prep_time)

        #calculate the waiting time
        waiting_time = order.exit_time - order.placement_time
        if order.arrival - order.exit_time < 0:
              pickup_delay = 0
        else:
              pickup_delay = order.arrival - order.exit_time
        print(f"{format_time(env.now)}: {order.name} exited at {round(env.now, 3)} minutes. Waiting time: {round(waiting_time, 3)} minutes and pickup delay: {pickup_delay}.")
        waiting_times.append(waiting_time)
        pickup_delays.append(pickup_delay)

        
#seting the numpy randome seed to 3
numpy.random.seed(3)
#createing an enviroment to run our simulation in
env = simpy.Environment()
#creating the resource for the queue
order_queue = simpy.Store(env)

#list of all the orders waiting time
waiting_times = []
pickup_delays = []

#add the 2 process in our environment
env.process(order_generator(env, order_queue))
env.process(process_orders(env, order_queue))

#run the environment for 60 steps
env.run(until=100)

#after the environment stops:

#calculate the average waiting time and print it
if len(waiting_times) == 0:
    avg_waiting_time = 0
else:
    avg_waiting_time = sum(waiting_times) / len(waiting_times)

if len(pickup_delays) == 0:
    avg_pickup_delay = 0
else:
    avg_pickup_delay = sum(pickup_delays) / len(pickup_delays)

#displaying results 
print(f'\nRESULTS:\nnumber of orders processed: {len(waiting_times)}\nAverage Waiting Time: {round(avg_waiting_time, 3)} minutes\nnAverage pickup delay: {round(avg_pickup_delay, 3)}')

00:00:41: order1 entered with prep time 5.201 minutes.
00:05:13: order2 entered with prep time 4.86 minutes.
00:05:13: order3 entered with prep time 2.537 minutes.
00:05:16: order4 entered with prep time 2.211 minutes.
00:05:53: order1 exited at 5.889 minutes. Waiting time: 0.0 minutes and pickup delay: 4.23846919661761.
00:05:56: order5 entered with prep time 4.441 minutes.
00:05:58: order6 entered with prep time 6.511 minutes.
00:05:58: order7 entered with prep time 5.274 minutes.
00:08:20: order8 entered with prep time 4.145 minutes.
00:09:55: order9 entered with prep time 4.323 minutes.
00:10:25: order10 entered with prep time 8.062 minutes.
00:10:25: order11 entered with prep time 3.964 minutes.
00:10:44: order2 exited at 10.749 minutes. Waiting time: 0.669 minutes and pickup delay: 3.0494691966176104.
00:13:17: order3 exited at 13.286 minutes. Waiting time: 5.529 minutes and pickup delay: 0.
00:15:04: order12 entered with prep time 16.918 minutes.
00:15:16: order13 entered with p

1 - FIFO – Multiple service Centers

In [101]:
class Order:
  def __init__(self, name, prep_time, arrival):
        self.name = name
        self.prep_time = prep_time
        self.arrival = arrival
        self.placement_time = None
        self.exit_time = None


def format_time(sim_time):
    hours, minutes = divmod(int(sim_time), 60)
    seconds = int((sim_time % 1) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#this function creates the orders and adds them to the queue
def order_generator(env, order_queue):
    order_count = 1
    while True:
        #set the minimum
        minimum = 2
        #generate a random preparation time (adding the 2 to make it the min)
        prep_time = round(minimum + numpy.random.exponential(scale=4.0, size=1)[0], 3)
        arrival = round(numpy.random.exponential(scale=4.0, size=1)[0], 3)
        #create an order with that preparation time
        order = Order(f"order{order_count}", prep_time, arrival)
        #increment the order counter
        order_count += 1
        
        #allowing some orders arrive at the same time
        #if this isn't satisfyed the next order will arive at the same time
        if (order_count % 4 != 0):
            #pauses this function for a random arrival time mimicking this part being idle
            yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])  
        #pushes the order in the queue
        yield order_queue.put(order)
        #gives feedback that an order have arrived
        order.placement_time = env.now
        print(f"{format_time(env.now)}: {order.name} entered with prep time {order.prep_time} minutes.")

#this function process orders
def process_orders(env, order_queue):
    while True:
        #get the order that is on the top of the queue
        order = yield order_queue.get()
        #printing when a new order is occupating a service center
        print(f"{format_time(env.now)}: {order.name} in service centers")
        #stores the time the order exists the queue
        order.exit_time = env.now
        #pauses this function for the preparation time, mimicking the service center creating the order
        yield env.timeout(order.prep_time)

        #calculate the waiting time
        waiting_time = order.exit_time - order.placement_time
        if order.arrival - order.exit_time < 0:
              pickup_delay = 0
        else:
              pickup_delay = order.arrival - order.exit_time
        print(f"{format_time(env.now)}: {order.name} exited at {round(env.now, 3)} minutes. Waiting time: {round(waiting_time, 3)} minutes and pickup delay: {pickup_delay}.")
        waiting_times.append(waiting_time)
        pickup_delays.append(pickup_delay)

        
#seting the numpy randome seed to 3
numpy.random.seed(3)
#createing an enviroment to run our simulation in
env = simpy.Environment()
#creating the resource for the queue
order_queue = simpy.Store(env)

#list of all the orders waiting time
waiting_times = []
pickup_delays = []
#number of service centers
SC = 3

#add the 2 process in our environment
env.process(order_generator(env, order_queue))

#for each service center add a 'process_orders' to the environment (they will run semitonically)
for sc in range(SC):
    env.process(process_orders(env, order_queue))

#run the environment for 60 steps
env.run(until=60)

#after the environment stops:

#calculate the average waiting time and print it
if len(waiting_times) == 0:
    avg_waiting_time = 0
else:
    avg_waiting_time = sum(waiting_times) / len(waiting_times)

if len(pickup_delays) == 0:
    avg_pickup_delay = 0
else:
    avg_pickup_delay = sum(pickup_delays) / len(pickup_delays)

#displaying results 
print(f'\nRESULTS:\nnumber of orders processed: {len(waiting_times)}\nAverage Waiting Time: {round(avg_waiting_time, 3)} minutes\nnAverage pickup delay: {round(avg_pickup_delay, 3)}')

00:00:41: order1 entered with prep time 5.201 minutes.
00:00:41: order1 in service centers
00:05:13: order2 entered with prep time 4.86 minutes.
00:05:13: order2 in service centers
00:05:13: order3 entered with prep time 2.537 minutes.
00:05:13: order3 in service centers
00:05:16: order4 entered with prep time 2.211 minutes.
00:05:53: order1 exited at 5.889 minutes. Waiting time: 0.0 minutes and pickup delay: 4.23846919661761.
00:05:53: order4 in service centers
00:05:56: order5 entered with prep time 4.441 minutes.
00:05:58: order6 entered with prep time 6.511 minutes.
00:05:58: order7 entered with prep time 5.274 minutes.
00:07:45: order3 exited at 7.757 minutes. Waiting time: 0.0 minutes and pickup delay: 0.
00:07:45: order5 in service centers
00:08:05: order4 exited at 8.1 minutes. Waiting time: 0.608 minutes and pickup delay: 0.
00:08:05: order6 in service centers
00:08:20: order8 entered with prep time 4.145 minutes.
00:09:55: order9 entered with prep time 4.323 minutes.
00:10:04

2 - FIFO – Limited Queue

In [104]:
#the order object that will be processed through the queue
class Order:
  def __init__(self, name, prep_time, arrival):
        self.name = name
        self.prep_time = prep_time
        self.arrival = arrival
        self.placement_time = None
        self.exit_time = None



def format_time(sim_time):
    hours, minutes = divmod(int(sim_time), 60)
    seconds = int((sim_time % 1) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#this function creates the orders and adds them to the queue
def order_generator(env, order_queue):
    order_count = 1
    

    while True:
        #set the minimum
        minimum = 2
        #generate a random preparation time (adding the 2 to make it the min)
        prep_time = round(minimum + numpy.random.exponential(scale=4.0, size=1)[0], 3)
        arrival = round(numpy.random.exponential(scale=4.0, size=1)[0], 3)
        #create an order with that preparation time
        order = Order(f"order{order_count}", prep_time, arrival)
        #if number of orders in the queue is less/equal the capacity
        if ((order_count-len(waiting_times)) < orderCap):
            #increment the order counter
            order_count += 1
        
            #allowing some orders arrive at the same time
            #if this isn't satisfyed the next order will arive at the same time
            if (order_count % 4 != 0):
                #pauses this function for a random arrival time mimicking this part being idle
                yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])
                
            #pushes the order in the queue
            yield order_queue.put(order)
            #gives feedback that an order have arrived
            order.placement_time = env.now
            print(f"{format_time(env.now)}: {order.name} entered with prep time {order.prep_time} minutes.")

        #otherwise
        else:
            #give feedback that the queue is full
            print(f"{format_time(env.now)}: queue full. {order.name} have been rejected")
            #pauses this function for a random arrival time mimicking this part being idle
            yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])


#this function process orders
def process_orders(env, order_queue):
    while True:
        #get the order that is on the top of the queue
        order = yield order_queue.get()
        #printing when a new order is occupating a service center
        print(f"{format_time(env.now)}: {order.name} in service centers")
        #stores the time the order exists the queue
        order.exit_time = env.now
        #pauses this function for the preparation time, mimicking the service center creating the order
        yield env.timeout(order.prep_time)

        #calculate the waiting time
        waiting_time = order.exit_time - order.placement_time
        if order.arrival - order.exit_time < 0:
              pickup_delay = 0
        else:
              pickup_delay = order.arrival - order.exit_time
        print(f"{format_time(env.now)}: {order.name} exited at {round(env.now, 3)} minutes. Waiting time: {round(waiting_time, 3)} minutes and pickup delay: {pickup_delay}.")
        waiting_times.append(waiting_time)
        pickup_delays.append(pickup_delay)

        
#seting the numpy randome seed to 3
numpy.random.seed(3)
#createing an enviroment to run our simulation in
env = simpy.Environment()
#creating the resource for the queue
order_queue = simpy.Store(env)

#list of all the orders waiting time
waiting_times = []
pickup_delays = []
#number of service centers
SC = 3
#set order capacity 
orderCap = 5

#add the 2 process in our environment
env.process(order_generator(env, order_queue))

#for each service center add a 'process_orders' to the environment (they will run semitonically)
for sc in range(SC):
    env.process(process_orders(env, order_queue))

#run the environment for 60 steps
env.run(until=60)

#after the environment stops:

#calculate the average waiting time and print it
if len(waiting_times) == 0:
    avg_waiting_time = 0
else:
    avg_waiting_time = sum(waiting_times) / len(waiting_times)

if len(pickup_delays) == 0:
    avg_pickup_delay = 0
else:
    avg_pickup_delay = sum(pickup_delays) / len(pickup_delays)

#displaying results 
print(f'\nRESULTS:\nnumber of orders processed: {len(waiting_times)}\nAverage Waiting Time: {round(avg_waiting_time, 3)} minutes\nnAverage pickup delay: {round(avg_pickup_delay, 3)}')

00:00:41: order1 entered with prep time 5.201 minutes.
00:00:41: order1 in service centers
00:05:13: order2 entered with prep time 4.86 minutes.
00:05:13: order2 in service centers
00:05:13: order3 entered with prep time 2.537 minutes.
00:05:13: order3 in service centers
00:05:16: order4 entered with prep time 2.211 minutes.
00:05:16: queue full. order5 have been rejected
00:05:53: order1 exited at 5.889 minutes. Waiting time: 0.0 minutes and pickup delay: 4.23846919661761.
00:05:53: order4 in service centers
00:05:58: order5 entered with prep time 6.511 minutes.
00:05:58: queue full. order6 have been rejected
00:07:03: queue full. order6 have been rejected
00:07:45: order3 exited at 7.757 minutes. Waiting time: 0.0 minutes and pickup delay: 0.
00:07:45: order5 in service centers
00:08:05: order4 exited at 8.1 minutes. Waiting time: 0.608 minutes and pickup delay: 0.
00:10:04: order2 exited at 10.08 minutes. Waiting time: 0.0 minutes and pickup delay: 3.7180961542984328.
00:11:14: orde

3 - Sorted Queue (priority queue)

In [48]:
class Order:
  def __init__(self, name, prep_time, arrival):
        self.name = name
        self.prep_time = prep_time
        self.arrival = arrival
        self.start_prep = arrival-prep_time
        self.placement_time = None
        self.exit_time = None


def format_time(sim_time):
    hours, minutes = divmod(int(sim_time), 60)
    seconds = int((sim_time % 1) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def pushing (order, order_queue):

  if len(order_queue.items) == 0:
      order_queue.put(order)
      return env.timeout(0)
  else:

      i=0
      for o in order_queue.items:
        if o.start_prep > order.start_prep:
            break
        i += 1
      if i == 0:
          order_queue.items = order_queue.items + [order]
      else:
          order_queue.items = order_queue.items[0:i] + [order] + order_queue.items[i:]

      return env.timeout(0)



def order_generator(env, order_queue):
    order_count = 1

    while True:
        minimum = 2
        prep_time = round(minimum + numpy.random.exponential(scale=4.0, size=1)[0], 3)
        arrival = round(numpy.random.exponential(scale=4.0, size=1)[0], 3)
        order = Order(f"order{order_count}", prep_time, arrival)
        if (len(order_queue.items) <= orderCap):
            order_count += 1

            #allowing some orders arrive at the same time
            if (order_count % 4 != 0):
                #pauses this function for a random placement time mimicking this part being idle
                yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])

            yield pushing(order, order_queue)
            order.placement_time = env.now
            print(f"{format_time(env.now)}: {order.name} entered with prep time {order.prep_time} minutes and estimated arrival time {order.arrival}")

        #otherwise
        else:
            print(f"{format_time(env.now)}: queue full. {order.name} have been rejected")
            #pauses this function for a random arrival time mimicking this part being idle
            yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])


#this function process orders
def process_orders(env, order_queue):
    while True:
        if (len(order_queue.items) > 0) and (round(env.now, 3) >= round(order_queue.items[0].start_prep, 3)):
          order = yield order_queue.get()
          
          print(f"{format_time(env.now)}: {order.name} in service centers, start preparation time: {round(order.start_prep, 3)}")

          # pauses this function for the preparation time, mimicking the service center creating the order
          yield env.timeout(order.prep_time)
          order.exit_time = env.now
          waiting_time = order.exit_time - order.placement_time
          if order.arrival - order.exit_time < 0:
              pickup_delay = 0
          else:
              pickup_delay = order.arrival - order.exit_time
          print(f"{format_time(env.now)}: {order.name} exited at {round(env.now, 3)} minutes. Waiting time: {round(waiting_time, 3)} minutes and pickup delay: {pickup_delay}.")
          waiting_times.append(waiting_time)
          pickup_delays.append(pickup_delay)

        else:
          yield env.timeout(0.00001)


numpy.random.seed(3)
env = simpy.Environment()
order_queue = simpy.Store(env)

waiting_times = []
pickup_delays = []
SC = 3
orderCap = 2


env.process(order_generator(env, order_queue))

#for each service center add a 'process_orders' to the environment (they will run semitonically)
#for sc in range(SC):
env.process(process_orders(env, order_queue))

env.run(until=60)

if len(waiting_times) == 0:
    avg_waiting_time = 0
else:
    avg_waiting_time = sum(waiting_times) / len(waiting_times)

if len(pickup_delays) == 0:
    avg_pickup_delay = 0
else:
    avg_pickup_delay = sum(pickup_delays) / len(pickup_delays)

#displaying results 
print(f'\nRESULTS:\nnumber of orders processed: {len(waiting_times)}\nAverage Waiting Time: {round(avg_waiting_time, 3)} minutes\nAverage pickup delay: {round(avg_pickup_delay, 3)}')

00:00:41: order1 entered with prep time 5.201 minutes and estimated arrival time 4.926
00:00:41: order1 in service centers, start preparation time: -0.275
00:05:13: order2 entered with prep time 4.86 minutes and estimated arrival time 8.938
00:05:13: order3 entered with prep time 2.537 minutes and estimated arrival time 0.929
00:05:16: order4 entered with prep time 2.211 minutes and estimated arrival time 2.325
00:05:16: queue full. order5 have been rejected
00:05:53: order1 exited at 5.889 minutes. Waiting time: 5.201 minutes and pickup delay: 0.
00:05:53: order2 in service centers, start preparation time: 4.078
00:05:58: order5 entered with prep time 6.511 minutes and estimated arrival time 3.575
00:05:58: queue full. order6 have been rejected
00:07:03: queue full. order6 have been rejected
00:08:12: queue full. order6 have been rejected
00:10:44: order2 exited at 10.749 minutes. Waiting time: 5.529 minutes and pickup delay: 0.
00:10:44: order3 in service centers, start preparation t

4 - Preparing at start_prep

In [47]:
class Order:
  def __init__(self, name, prep_time, arrival):
        self.name = name
        self.prep_time = prep_time
        self.arrival = arrival
        self.placement_time = None
        self.exit_time = None



def format_time(sim_time):
    hours, minutes = divmod(int(sim_time), 60)
    seconds = int((sim_time % 1) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#this function creates the orders and adds them to the queue
def order_generator(env, order_queue):
    order_count = 1
    

    while True:
        #set the minimum
        minimum = 2
        #generate a random preparation time (adding the 2 to make it the min)
        prep_time = round(minimum + numpy.random.exponential(scale=4.0, size=1)[0], 3)
        arrival = round(numpy.random.exponential(scale=4.0, size=1)[0], 3)
        #create an order with that preparation time
        order = Order(f"order{order_count}", prep_time, arrival)
        
        #increment the order counter
        order_count += 1
    
        #allowing some orders arrive at the same time
        #if this isn't satisfyed the next order will arive at the same time
        if (order_count % 4 != 0):
            #pauses this function for a random arrival time mimicking this part being idle
            yield env.timeout(numpy.random.exponential(scale=2.0, size=1)[0])
            
        #pushes the order in the queue
        yield order_queue.put(simpy.PriorityItem(prep_time, order))
        #gives feedback that an order have arrived
        order.placement_time = env.now
        print(f"{format_time(env.now)}: {order.name} entered with prep time {order.prep_time} minutes.")


#this function process orders
def process_orders(env, order_queue):
    while True:
        #get the order that is on the top of the queue
        order = yield order_queue.get()
        order = order[1]
        #printing when a new order is occupating a service center
        print(f"{format_time(env.now)}: {order.name} in service centers")
        #stores the time the order exists the queue
        order.exit_time = env.now
        #pauses this function for the preparation time, mimicking the service center creating the order
        yield env.timeout(order.prep_time)

        #calculate the waiting time
        waiting_time = order.exit_time - order.placement_time
        if order.arrival - order.exit_time < 0:
              pickup_delay = 0
        else:
              pickup_delay = order.arrival - order.exit_time
        print(f"{format_time(env.now)}: {order.name} exited at {round(env.now, 3)} minutes. Waiting time: {round(waiting_time, 3)} minutes and pickup delay: {pickup_delay}.")
        waiting_times.append(waiting_time)
        pickup_delays.append(pickup_delay)

        
#seting the numpy randome seed to 3
numpy.random.seed(3)
#createing an enviroment to run our simulation in
env = simpy.Environment()

#list of all the orders waiting time
waiting_times = []
pickup_delays = []
#number of service centers
SC = 1
#set order capacity 
orderCap = 5

#creating the resource for the queue
order_queue = simpy.PriorityStore(env, orderCap)
#add the 2 process in our environment
env.process(order_generator(env, order_queue))

#for each service center add a 'process_orders' to the environment (they will run semitonically)
# for sc in range(SC):
#     env.process(process_orders(env, order_queue))
env.process(process_orders(env, order_queue))
#run the environment for 60 steps
env.run(until=60)

#after the environment stops:

#calculate the average waiting time and print it
if len(waiting_times) == 0:
    avg_waiting_time = 0
else:
    avg_waiting_time = sum(waiting_times) / len(waiting_times)

if len(pickup_delays) == 0:
    avg_pickup_delay = 0
else:
    avg_pickup_delay = sum(pickup_delays) / len(pickup_delays)

#displaying results 
print(f'\nRESULTS:\nnumber of orders processed: {len(waiting_times)}\nAverage Waiting Time: {round(avg_waiting_time, 3)} minutes\nAverage pickup delay: {round(avg_pickup_delay, 3)}')

00:00:41: order1 entered with prep time 5.201 minutes.
00:00:41: order1 in service centers
00:05:13: order2 entered with prep time 4.86 minutes.
00:05:13: order3 entered with prep time 2.537 minutes.
00:05:16: order4 entered with prep time 2.211 minutes.
00:05:53: order1 exited at 5.889 minutes. Waiting time: 0.0 minutes and pickup delay: 4.23846919661761.
00:05:53: order4 in service centers
00:05:56: order5 entered with prep time 4.441 minutes.
00:05:58: order6 entered with prep time 6.511 minutes.
00:05:58: order7 entered with prep time 5.274 minutes.
00:08:05: order4 exited at 8.1 minutes. Waiting time: 0.608 minutes and pickup delay: 0.
00:08:05: order3 in service centers
00:08:20: order8 entered with prep time 4.145 minutes.
00:10:38: order3 exited at 10.637 minutes. Waiting time: 2.88 minutes and pickup delay: 0.
00:10:38: order8 in service centers
00:10:38: order9 entered with prep time 4.323 minutes.
00:14:46: order8 exited at 14.782 minutes. Waiting time: 2.292 minutes and pic